## PageRank: A Study on Stanford's Web Network

### Importing necessary libraries

In [1]:
# import necessary libraries
import numpy as np
from scipy import sparse
from numpy.linalg import norm
import time
np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
# method to parse the data. Obviously, storing all that information
# naively in a numpy array would lead to memory error. Thus, sparse
# matrix is used.
def load_dataset(data):
    # list to store the origin pages,
    origin_pages = []
    # list to store the destination pages,
    destination_pages = []
    # dictionary having as a key a webpage and as value
    # the number of its invoming edges
    dict_out = {}
    dict_in = {}
    with open(data, 'r') as f:
        # for each line of the initial data file
        for line in f.readlines():
            # get the item of the first column
            # and substract one. This is done because the ids of webpages
            # begin from 1 but the indexes for matrices begine from 0. So
            # 1 will stand for 0.
            origin_page = int(line.split()[0]) - 1
            # Apply the same logic as above for the element of the second
            # column.
            destination_page = int(line.split()[1]) - 1
            # append the the origin page to the relative list
            origin_pages.append(origin_page)
            # append the the destination page to the relative list
            destination_pages.append(destination_page)
            # fill the dictionary by counting how many times
            # each destination-page has appeared.
            if origin_page not in dict_out.keys():
                dict_out[origin_page] = 1
            else:
                dict_out[origin_page] += 1
            if destination_page not in dict_in.keys():
                dict_in[destination_page] = [origin_page]
            else:
                dict_in[destination_page].append(origin_page)
            
    # compute number of edges
    edges = len(origin_pages)
    # compute number of nodes
    nodes = len(set(origin_pages) | set(destination_pages))
    
    for node in range(nodes):  
        if node not in dict_in.keys():
            dict_in[node] = []

    # initialize a row-based linked list sparse matrix
    # that matrix will be used for the Gauss-Seidel implementation
    # of the PageRank
    D = sparse.lil_matrix((nodes, 1))
    for key in dict_out.keys():
        D[key] = 1/dict_out[key]
    # fill the connectivity matrix, adjacency matrix. A sparse structure 
    # is used. That way the implementation is extremely memory efficient.
    csr_m = sparse.csr_matrix(([1]*edges,
                               (destination_pages, origin_pages)),
                              shape=(nodes, nodes))
    # return the two created matrices
    return csr_m, D, dict_in

In [3]:
def pagerank_galgo(G, alpha=0.85, tol=10**-8):
    # The matrix G is the adjacency matrix with dead-ends and closed communities or spidertraps.
    # Those problems will be tackled during the iterations of the algorithm. In other words, we do  
    # not pre-adjust the matrix P. Instead we reinsert leaked page-rank back into computation.
    
    # compute the number of nodes, the matrix is symmetric
    n = G.shape[0]
    # compute the ratio of the out degree of each node
    # to the alpha
    out_alpha = G.sum(axis=0).T/alpha
    r = np.ones((n,1))/n
    error = 1
    t = 0
    while error > tol:
        t += 1
        r_new = G.dot((r/out_alpha))
        #  L≤alpha due to dead-ends
        L = r_new.sum()
        # re-insert leaked page-rank
        # now all ranks add to one
        r_new += (1-L)/n
        # compute the error as the euclidean norm of the
        # previous ranks and the new ranks
        error = np.linalg.norm(r-r_new)
        # store the new ranks as the ranks of the current
        # iteration
        if t == 2:
            # list to return the nodes that converged from the
            # second iteration
            list_conv = []
            for i in range(r.shape[0]):
                if np.linalg.norm(r[i]-r_new[i]) < tol:
                    list_conv.append(i)
        r = r_new
    return r, t, list_conv

In [4]:
def pagerank_gs(tol, G, D, dict_in, alpha):
    # in that approach a linear system approach is used to solve
    # get the ranks of each webpage
    
    # the initial solution is again given from the uniform distribution
    # i.e. we give equal weights to each webpage
    x_old = np.ones(G.shape[0])/G.shape[0]
    t = 0
    error = 1
    b = 1/G.shape[0]
    D = alpha*D
    x = x_old.copy()
    
    # apply the element-wise formula for the Gauss–Seidel method for 
    # the pagerank. That formula looks a lot like the Jacobi formula 
    # and it is the same with SOR for omega equals to 1. As input it
    # is used the initial matrix P without any preadjustments as well
    # as a dictionary showing all the in-edged of each webpage. In general,
    # the following implementation scheme is based on the paper:
    # https://web.ece.ucsb.edu/~hespanha/published/pagerank.pdf
    while error > tol:
        t += 1
        for i in range(G.shape[0]):

            sum_before = 0
            sum_after = 0
            for j in dict_in[i]:
                if j < i:
                    sum_before += D.data[j][0] * x[j]

                if j > i:
                    sum_after += D.data[j][0] * x_old[j]

            x[i] = b + sum_before + sum_after

        error = np.linalg.norm(x-x_old)/np.linalg.norm(x)
        if t == 2:
            # list to return the nodes that converged from the
            # second iteration
            list_conv = []
            for i in range(x.shape[0]):
                if np.linalg.norm(x[i]-x_old[i]) < tol:
                    list_conv.append(i)

        print('Iteration:', t, '==> Relative Error:', error)
        x_old = x.copy()

    return x, list_conv

### Importing Dataset

In [9]:
csr_m, D, dict_in = load_dataset("./Dataset/stanweb.dat")

### Question 1a

#### i. Google's Algorithm - Power method

In [18]:
start_time = time.time()
pr, t, list_conv = pagerank_galgo(csr_m)
print("Time of Pagerank with Google's Algorithm:",
      time.time() - start_time, "seconds")
print ("Number of iterations:", t)
mat_prg = np.asarray(pr).ravel()

Time of Pagerank with Google's Algorithm: 6.182928085327148 seconds
Number of iterations: 72


#### ii. Solution of the corresponding system - Gauss Seidel

In [19]:
start_time = time.time()
gs, list_convgs = pagerank_gs(10**-8, csr_m, D, dict_in, 0.85)
print("Time of Pagerank with Gauss-Seidel:",
      time.time() - start_time, "seconds")
mat_prgs = np.asarray(gs).ravel()

Iteration: 1 ==> Relative Error: 0.997605864804151
Iteration: 2 ==> Relative Error: 0.4054109826036649
Iteration: 3 ==> Relative Error: 0.22047373339942417
Iteration: 4 ==> Relative Error: 0.14008157663540255
Iteration: 5 ==> Relative Error: 0.0916579942469785
Iteration: 6 ==> Relative Error: 0.06208189402282477
Iteration: 7 ==> Relative Error: 0.0428958007110852
Iteration: 8 ==> Relative Error: 0.03007122562269499
Iteration: 9 ==> Relative Error: 0.02130464756876604
Iteration: 10 ==> Relative Error: 0.015210011046644085
Iteration: 11 ==> Relative Error: 0.010921248126796736
Iteration: 12 ==> Relative Error: 0.007874909962757961
Iteration: 13 ==> Relative Error: 0.005696540133272372
Iteration: 14 ==> Relative Error: 0.0041307516243636435
Iteration: 15 ==> Relative Error: 0.003000912867833307
Iteration: 16 ==> Relative Error: 0.0021832470941981234
Iteration: 17 ==> Relative Error: 0.0015901745892285591
Iteration: 18 ==> Relative Error: 0.0011592462668742194
Iteration: 19 ==> Relative Er

#### **Results**

The Power method appears to be substantially quicker than the Gauss-Seidel version. Both strategies provide different outcomes. They are distinguished by 121562 rankings.

In [21]:
indices_pr = mat_prg.argsort()[-len(mat_prg):][::-1]
indices_prs = mat_prgs.argsort()[-len(mat_prgs):][::-1]
print("The two aforementioned methods differ in:",
      np.sum(indices_prs[:] != indices_pr[:]), "ranks.")

The two aforementioned methods differ in: 121652 ranks.


### Question 1b

In [23]:
start_time = time.time()
pr_99, t, list_conv99 = pagerank_galgo(csr_m, 0.99, 10**-8)
print("Time of Pagerank with Google's Algorithm:",
      time.time() - start_time, "seconds")
print ("Number of iterations:", t)
mat_prg_99 = np.asarray(pr_99).ravel()

Time of Pagerank with Google's Algorithm: 37.51247763633728 seconds
Number of iterations: 1141


In [24]:
start_time = time.time()
gs_99, list_convgs99 = pagerank_gs(10**-8, csr_m, D, dict_in, 0.99)

Iteration: 1 ==> Relative Error: 0.9981678259340745
Iteration: 2 ==> Relative Error: 0.47185982372499896
Iteration: 3 ==> Relative Error: 0.3100832127531775
Iteration: 4 ==> Relative Error: 0.23528133099965182
Iteration: 5 ==> Relative Error: 0.18625685865046143
Iteration: 6 ==> Relative Error: 0.15353070497237115
Iteration: 7 ==> Relative Error: 0.1300477866165402
Iteration: 8 ==> Relative Error: 0.11252370771418573
Iteration: 9 ==> Relative Error: 0.09899702839915098
Iteration: 10 ==> Relative Error: 0.0882611511428183
Iteration: 11 ==> Relative Error: 0.07954383560946221
Iteration: 12 ==> Relative Error: 0.0723243491142378
Iteration: 13 ==> Relative Error: 0.06624868022544683
Iteration: 14 ==> Relative Error: 0.061063168400621017
Iteration: 15 ==> Relative Error: 0.05658458227347333
Iteration: 16 ==> Relative Error: 0.05267618438552044
Iteration: 17 ==> Relative Error: 0.049235129342435256
Iteration: 18 ==> Relative Error: 0.046181384190046776
Iteration: 19 ==> Relative Error: 0.043

Iteration: 150 ==> Relative Error: 0.0016229460773825486
Iteration: 151 ==> Relative Error: 0.0015918920184666533
Iteration: 152 ==> Relative Error: 0.0015614400127057985
Iteration: 153 ==> Relative Error: 0.0015315795864559558
Iteration: 154 ==> Relative Error: 0.0015022975635218454
Iteration: 155 ==> Relative Error: 0.0014735832964369929
Iteration: 156 ==> Relative Error: 0.0014454240513175204
Iteration: 157 ==> Relative Error: 0.0014178099672102799
Iteration: 158 ==> Relative Error: 0.0013907287943163583
Iteration: 159 ==> Relative Error: 0.0013641710776691252
Iteration: 160 ==> Relative Error: 0.0013381253012679111
Iteration: 161 ==> Relative Error: 0.0013125823434212683
Iteration: 162 ==> Relative Error: 0.0012875307133251405
Iteration: 163 ==> Relative Error: 0.0012629620448718502
Iteration: 164 ==> Relative Error: 0.0012388657580448778
Iteration: 165 ==> Relative Error: 0.0012152333726132285
Iteration: 166 ==> Relative Error: 0.0011920546520422758
Iteration: 167 ==> Relative Err

Iteration: 294 ==> Relative Error: 0.00010115292431828253
Iteration: 295 ==> Relative Error: 9.921426585099823e-05
Iteration: 296 ==> Relative Error: 9.731255854994186e-05
Iteration: 297 ==> Relative Error: 9.544726200060094e-05
Iteration: 298 ==> Relative Error: 9.361752341250952e-05
Iteration: 299 ==> Relative Error: 9.182282441068077e-05
Iteration: 300 ==> Relative Error: 9.006234524799712e-05
Iteration: 301 ==> Relative Error: 8.833558787579323e-05
Iteration: 302 ==> Relative Error: 8.664175327951878e-05
Iteration: 303 ==> Relative Error: 8.498037023844864e-05
Iteration: 304 ==> Relative Error: 8.335067449187547e-05
Iteration: 305 ==> Relative Error: 8.175220677212855e-05
Iteration: 306 ==> Relative Error: 8.018422763243356e-05
Iteration: 307 ==> Relative Error: 7.864630091995653e-05
Iteration: 308 ==> Relative Error: 7.713771195002425e-05
Iteration: 309 ==> Relative Error: 7.56580426467011e-05
Iteration: 310 ==> Relative Error: 7.420660557608455e-05
Iteration: 311 ==> Relative Err

Iteration: 438 ==> Relative Error: 6.180822705457954e-06
Iteration: 439 ==> Relative Error: 6.0618308838608926e-06
Iteration: 440 ==> Relative Error: 5.94511326037879e-06
Iteration: 441 ==> Relative Error: 5.830658786261561e-06
Iteration: 442 ==> Relative Error: 5.718391478160739e-06
Iteration: 443 ==> Relative Error: 5.6083013697258974e-06
Iteration: 444 ==> Relative Error: 5.500314947829089e-06
Iteration: 445 ==> Relative Error: 5.394423020873723e-06
Iteration: 446 ==> Relative Error: 5.2905543070533954e-06
Iteration: 447 ==> Relative Error: 5.188700560709678e-06
Iteration: 448 ==> Relative Error: 5.088792673031303e-06
Iteration: 449 ==> Relative Error: 4.9908232233030995e-06
Iteration: 450 ==> Relative Error: 4.894725264799044e-06
Iteration: 451 ==> Relative Error: 4.800492147635945e-06
Iteration: 452 ==> Relative Error: 4.7080588632953666e-06
Iteration: 453 ==> Relative Error: 4.617419594189052e-06
Iteration: 454 ==> Relative Error: 4.528511409156216e-06
Iteration: 455 ==> Relative

Iteration: 581 ==> Relative Error: 3.840526151567064e-07
Iteration: 582 ==> Relative Error: 3.766701068039075e-07
Iteration: 583 ==> Relative Error: 3.694331967064742e-07
Iteration: 584 ==> Relative Error: 3.6233203994270704e-07
Iteration: 585 ==> Relative Error: 3.553709869811931e-07
Iteration: 586 ==> Relative Error: 3.485404478984577e-07
Iteration: 587 ==> Relative Error: 3.418447270822351e-07
Iteration: 588 ==> Relative Error: 3.352744824375728e-07
Iteration: 589 ==> Relative Error: 3.2883397085707427e-07
Iteration: 590 ==> Relative Error: 3.2251409249506864e-07
Iteration: 591 ==> Relative Error: 3.163190549847293e-07
Iteration: 592 ==> Relative Error: 3.1023999118415425e-07
Iteration: 593 ==> Relative Error: 3.0428106154684903e-07
Iteration: 594 ==> Relative Error: 2.9843362871199486e-07
Iteration: 595 ==> Relative Error: 2.9270180098158115e-07
Iteration: 596 ==> Relative Error: 2.870771629295234e-07
Iteration: 597 ==> Relative Error: 2.815637730676321e-07
Iteration: 598 ==> Relat

Iteration: 724 ==> Relative Error: 2.4093384196988265e-08
Iteration: 725 ==> Relative Error: 2.3632939612205604e-08
Iteration: 726 ==> Relative Error: 2.3180563651309796e-08
Iteration: 727 ==> Relative Error: 2.273760667850957e-08
Iteration: 728 ==> Relative Error: 2.230239836481767e-08
Iteration: 729 ==> Relative Error: 2.187626358947585e-08
Iteration: 730 ==> Relative Error: 2.1457570376321136e-08
Iteration: 731 ==> Relative Error: 2.1047617856330426e-08
Iteration: 732 ==> Relative Error: 2.064481192588485e-08
Iteration: 733 ==> Relative Error: 2.025042600386338e-08
Iteration: 734 ==> Relative Error: 1.9862903132810587e-08
Iteration: 735 ==> Relative Error: 1.9483491976476596e-08
Iteration: 736 ==> Relative Error: 1.911067106799162e-08
Iteration: 737 ==> Relative Error: 1.8745665048035662e-08
Iteration: 738 ==> Relative Error: 1.8386987499672088e-08
Iteration: 739 ==> Relative Error: 1.8035838586962745e-08
Iteration: 740 ==> Relative Error: 1.769076654012087e-08
Iteration: 741 ==> Re

In [31]:
print("Time of Pagerank with Gauss-Seidel:",
      time.time() - start_time, "seconds")
mat_prgs_99 = np.asarray(gs_99).ravel()

Time of Pagerank with Gauss-Seidel: 3924.5643560886383 seconds


#### Results

Both approaches converge more slowly when alpha=0.99. The Gauss-Seidel method for iterative system solution converges substantially slower than that time. Again, the power approach appears to be considerably quicker than the Gauss-Seidel version. The order of the first 50 nodes has altered. Except for the first most significant node, which remains at 89072, all of the others have shifted positions.

In [32]:
indices_pr_99 = mat_prg_99.argsort()[-len(mat_prg):][::-1]
indices_prs_99 = mat_prgs_99.argsort()[-len(mat_prg):][::-1]
print("The two aforementioned methods differ in:",
      sum(indices_prs_99[:] != indices_pr_99[:]), "ranks.")

The two aforementioned methods differ in: 138038 ranks.


In [33]:
print("We observe differences in:",
      sum(indices_pr[:50] != indices_pr_99[:50]),
      "ranks (power method).")

We observe differences in: 49 ranks (power method).


In [34]:
print("We observe differences in:",
      sum(indices_prs[:50] != indices_prs_99[:50]),
      "ranks (Gauss Seidel).")

We observe differences in: 49 ranks (Gauss Seidel).


### Question 1c

We observe that the components of $π$ that correspond to non important converge faster, as it is shown below.

In [35]:
print("When we use the power method",
      len(list_conv),
      "components of π converge at the second iteration.")

When we use the power method 42904 components of π converge at the second iteration.


In [36]:
counter = 0
for node in list_conv:
    if node in indices_pr[:1000]:
        counter += 1
print("From those components of π that converged faster",
      counter,
      "component(s) correspond to the top (1000) ranked nodes (Power method).")

From those components of π that converged faster 0 component(s) correspond to the top (1000) ranked nodes (Power method).


In [37]:
print("When we find π through the solution of the linear system",
      len(list_convgs),
      "components of π converge at the second iteration.")

When we find π through the solution of the linear system 34367 components of π converge at the second iteration.


In [38]:
counter = 0
for node in list_convgs:
    if node in indices_prs[:1000]:
        counter += 1
print("From those components of π that converged faster", counter,
      "component(s) correspond to the top (1000) ranked nodes (Gauss Seidel).")

From those components of π that converged faster 1 component(s) correspond to the top (1000) ranked nodes (Gauss Seidel).


### Question 2a

In [39]:
# method to parse the data. Obviously, storing all that information
# naively in a numpy array would lead to memory error. Thus, sparse
# matrix is used.
def load_dataset1(data):
    # list to store the origin pages,
    origin_pages = []
    # list to store the destination pages,
    destination_pages = []
    with open(data, 'r') as f:
        # for each line of the initial data file
        for line in f.readlines():
            # get the item of the first column
            # and substract one. This is done because the ids of webpages
            # begin from 1 but the indexes for matrices begine from 0. So
            # 1 will stand for 0.
            origin_page = int(line.split()[0]) - 1
            # Apply the same logic as above for the element of the second
            # column.
            destination_page = int(line.split()[1]) - 1
            # append the the from page to the relative list
            origin_pages.append(origin_page)
            # append the the to page to the relative list
            destination_pages.append(destination_page)
            
    # compute number of edges
    edges = len(origin_pages)
    # compute number of nodes
    nodes = len(set(origin_pages) | set(destination_pages))
    

    # fill the connectivity matrix, adjacency matrix. A sparse structure 
    # is used. That way the implementation is extremely memory efficient.
    # increase the size of the array by introducing a dangling node, i.e.
    # X
    csr_m = sparse.csr_matrix(([1]*edges,
                               (destination_pages, origin_pages)),
                              shape=(nodes + 1, nodes + 1))
    # return the adjacency sparse matrix
    return csr_m

In [40]:
csr_m1 = load_dataset1("./Dataset/stanweb.dat")
pr1, t1, list_conv1 = pagerank_galgo(csr_m1)

#### **Results**

We see that the PageRanks of the older pages affected by the addition of the new page stay nearly untouched, despite the fact that they are in no way related to the newly added page. Furthermore, we should keep in mind that the number of nodes is really massive. The PageRank of the newly uploaded page is anticipated to be computed using the PageRank formula. It is really just (1 - alpha)/n = 0.15/281904.

In [41]:
pageRank_X = np.asarray(pr1[csr_m1.shape[0] - 1]).ravel()[0]
print("The PageRank of page X is:", pageRank_X)

The PageRank of page X is: 5.333658781475117e-07


In [42]:
norm_diff = np.linalg.norm(np.asarray(pr[:csr_m.shape[0]].ravel()) - np.asarray(pr1[:csr_m1.shape[0] - 1].ravel()))
print("Change in PageRank of the other pages due to the addition of the new page:", norm_diff)

mat_prg = np.asarray(pr).ravel()
mat_prg1 = np.asarray(pr1[:csr_m1.shape[0] - 1]).ravel()

indices_pr = mat_prg.argsort()[-len(mat_prg):][::-1]
indices_pr1 = mat_prg1.argsort()[-len(mat_prg1):][::-1]
print("Changes in:",
      np.sum(indices_pr[:] != indices_pr1[:]), "ranks.")

Change in PageRank of the other pages due to the addition of the new page: 1.2048086354849136e-08
Changes in: 50570 ranks.


### Question 2b

In [43]:
# method to parse the data. Obviously, storing all that information
# naively in a numpy array would lead to memory error. Thus, sparse
# matrix is used.
def load_dataset2(data):
    # list to store the origin pages,
    from_pages = []
    # list to store the destination pages,
    to_pages = []
    with open(data, 'r') as f:
        # for each line of the initial data file
        for line in f.readlines():
            # get the item of the first column
            # and substract one. This is done because the ids of webpages
            # begin from 1 but the indexes for matrices begine from 0. So
            # 1 will stand for 0.
            from_page = int(line.split()[0]) - 1
            # Apply the same logic as above for the element of the second
            # column.
            to_page = int(line.split()[1]) - 1
            # append the the from page to the relative list
            from_pages.append(from_page)
            # append the the to page to the relative list
            to_pages.append(to_page)
            
    # compute number of edges
    edges = len(from_pages)
    # compute number of nodes
    nodes = len(set(from_pages) | set(to_pages))
    
    # add a new edge Y -> X
    from_pages.append(nodes)
    to_pages.append(nodes + 1)
    # increase the number of nodes by two for X, Y
    nodes += 2
    # increase the number of edges by one
    edges += 1
    
    # fill the connectivity matrix, adjacency matrix. A sparse structure 
    # is used. That way the implementation is extremely memory efficient.
    csr_m = sparse.csr_matrix(([1]*edges,
                               (to_pages, from_pages)),
                              shape=(nodes, nodes))
    # return the adjacency sparse matrix
    return csr_m

In [44]:
csr_m2 = load_dataset2("./Dataset/stanweb.dat")
pr2, t2, list_conv2 = pagerank_galgo(csr_m2)

#### **Results**

We see that the PageRanks of older pages changed more throughout that time period owing to the inclusion of new pages. Furthermore, we should keep in mind that the number of nodes is really enormous. The PageRank of the newly uploaded page is anticipated to be computed using the PageRank formula. The PageRank of X has clearly improved, whereas the PageRank of Y is the same as the PageRank of X in the preceding question.

In [45]:
pageRank_Xb = np.asarray(pr2[csr_m2.shape[0] - 1]).ravel()[0]
print("The PageRank of page X is:", pageRank_Xb)

The PageRank of page X is: 9.867259009702931e-07


In [46]:
pageRank_Yb = np.asarray(pr2[csr_m2.shape[0] - 2]).ravel()[0]
print("The PageRank of page Y is:", pageRank_Yb)

The PageRank of page Y is: 5.333653518615822e-07


In [47]:
norm_diff = np.linalg.norm(np.asarray(pr[:csr_m.shape[0]].ravel()) - np.asarray(pr2[:csr_m2.shape[0] - 2].ravel()))
print("Change in PageRank of the other pages due to the addition of the new page:", norm_diff)

mat_prg = np.asarray(pr).ravel()
mat_prg2 = np.asarray(pr2[:csr_m2.shape[0] - 2]).ravel()

indices_pr = mat_prg.argsort()[-len(mat_prg):][::-1]
indices_pr2 = mat_prg2.argsort()[-len(mat_prg2):][::-1]
print("Changes in:",
      np.sum(indices_pr[:] != indices_pr2[:]), "ranks.")

Change in PageRank of the other pages due to the addition of the new page: 3.433701367463661e-08
Changes in: 51477 ranks.


### Question 2c

In [48]:
# method to parse the data. Obviously, storing all that information
# naively in a numpy array would lead to memory error. Thus, sparse
# matrix is used.
def load_dataset3(data):
    # list to store the origin pages,
    # i.e. the pages showing to other pages
    from_pages = []
    # list to store the destination pages,
    # i.e. the pages being showed by other pages
    to_pages = []
    with open(data, 'r') as f:
        # for each line of the initial data file
        for line in f.readlines():
            # get the item of the first column
            # and substract one. This is done because the ids of webpages
            # begin from 1 but the indexes for matrices begine from 0. So
            # 1 will stand for 0.
            from_page = int(line.split()[0]) - 1
            # Apply the same logic as above for the element of the second
            # column.
            to_page = int(line.split()[1]) - 1
            # append the the from page to the relative list
            from_pages.append(from_page)
            # append the the to page to the relative list
            to_pages.append(to_page)
            
    # compute number of edges
    edges = len(from_pages)
    # compute number of nodes
    nodes = len(set(from_pages) | set(to_pages))
    
    # add a new edge Y -> X
    from_pages.append(nodes)
    to_pages.append(nodes + 2)
    
    # add a new edge Z -> X
    from_pages.append(nodes + 1)
    to_pages.append(nodes + 2)

    # increases as needed the number of nodes and edges
    nodes += 3
    edges += 2
    
    # fill the connectivity matrix, adjacency matrix. A sparse structure 
    # is used. That way the implementation is extremely memory efficient.
    csr_m = sparse.csr_matrix(([1]*edges,
                               (to_pages, from_pages)),
                              shape=(nodes, nodes))
    # return the adjacency sparse matrix
    return csr_m

In [49]:
csr_m3 = load_dataset3("./Dataset/stanweb.dat")
pr3, t3, list_conv3 = pagerank_galgo(csr_m3)

#### Results

In order to to maximize the PageRank of X, both Y and Z should point only to X, according to logic behind PageRank. This is implemented and the results are presented below. Obviously, the PageRank of X is now again improved.

In [50]:
pageRank_Xc = np.asarray(pr3[csr_m3.shape[0] - 1]).ravel()[0]
print("The PageRank of page X is:", pageRank_Xc)

The PageRank of page X is: 1.4400850291097994e-06


In [51]:
pageRank_Yc = np.asarray(pr3[csr_m3.shape[0] - 2]).ravel()[0]
print("The PageRank of page Y is:", pageRank_Yc)

The PageRank of page Y is: 5.333648255766965e-07


In [52]:
pageRank_Zc = np.asarray(pr3[csr_m3.shape[0] - 3]).ravel()[0]
print("The PageRank of page Z is:", pageRank_Zc)

The PageRank of page Z is: 5.333648255766965e-07


In [53]:
norm_diff = np.linalg.norm(np.asarray(pr[:csr_m.shape[0]].ravel()) - np.asarray(pr3[:csr_m3.shape[0] - 3].ravel()))
print("Changes in PageRank of the other pages due to the addition of the new page:", norm_diff)

mat_prg = np.asarray(pr).ravel()
mat_prg3 = np.asarray(pr3[:csr_m3.shape[0] - 3]).ravel()

indices_pr = mat_prg.argsort()[-len(mat_prg):][::-1]
indices_pr3 = mat_prg3.argsort()[-len(mat_prg3):][::-1]
print("Changes in:",
      np.sum(indices_pr[:] != indices_pr3[:]), "ranks.")

Changes in PageRank of the other pages due to the addition of the new page: 5.6625897409508286e-08
Changes in: 52911 ranks.


### Question2d

In [54]:
# determine top 100 - most popular pages - here as popular are considered
# pages with high PageRank, because PageRank can be seen as a measure of
# popularity
top_100 = indices_pr[0:100]

# determine top 100 - most popular pages - here as popular are considered
# pages with high in-degree
list_degree = np.asarray([len(dict_in[x]) for x in sorted(list(dict_in.keys()))])
indices_degree = list_degree.argsort()[-len(list_degree):][::-1]
popular_100 = list_degree[0:100]

In [55]:
# method to parse the data. Obviously, storing all that information
# naively in a numpy array would lead to memory error. Thus, sparse
# matrix is used.
def load_dataset4(data, top100):
    # list to store the origin pages,
    # i.e. the pages showing to other pages
    from_pages = []
    # list to store the destination pages,
    # i.e. the pages being showed by other pages
    to_pages = []
    with open(data, 'r') as f:
        # for each line of the initial data file
        for line in f.readlines():
            # get the item of the first column
            # and substract one. This is done because the ids of webpages
            # begin from 1 but the indexes for matrices begine from 0. So
            # 1 will stand for 0.
            from_page = int(line.split()[0]) - 1
            # Apply the same logic as above for the element of the second
            # column.
            to_page = int(line.split()[1]) - 1
            # append the the from page to the relative list
            from_pages.append(from_page)
            # append the the to page to the relative list
            to_pages.append(to_page)
            
    # compute number of edges
    edges = len(from_pages)
    # compute number of nodes
    nodes = len(set(from_pages) | set(to_pages))
    
    # add a new edge Y -> X
    from_pages.append(nodes)
    to_pages.append(nodes + 2)
    
    # add a new edge Z -> X
    from_pages.append(nodes + 1)
    to_pages.append(nodes + 2)
    
    # add links from X to older, popular pages
    for node in top_100:
        from_pages.append(nodes + 2)
        to_pages.append(node)
    
    # increase number of nodes and edges as needed
    nodes += 3
    edges += 102
    
    # fill the connectivity matrix, adjacency matrix. A sparse structure 
    # is used. That way the implementation is extremely memory efficient.
    csr_m = sparse.csr_matrix(([1]*edges,
                               (to_pages, from_pages)),
                              shape=(nodes, nodes))
    # return the adjacency sparse matrix
    return csr_m

In [56]:
# method to parse the data. Obviously, storing all that information
# naively in a numpy array would lead to memory error. Thus, sparse
# matrix is used.
def load_dataset5(data, top100):
    # list to store the origin pages,
    # i.e. the pages showing to other pages
    from_pages = []
    # list to store the destination pages,
    # i.e. the pages being showed by other pages
    to_pages = []
    with open(data, 'r') as f:
        # for each line of the initial data file
        for line in f.readlines():
            # get the item of the first column
            # and substract one. This is done because the ids of webpages
            # begin from 1 but the indexes for matrices begine from 0. So
            # 1 will stand for 0.
            from_page = int(line.split()[0]) - 1
            # Apply the same logic as above for the element of the second
            # column.
            to_page = int(line.split()[1]) - 1
            # append the the from page to the relative list
            from_pages.append(from_page)
            # append the the to page to the relative list
            to_pages.append(to_page)
            
    # compute number of edges
    edges = len(from_pages)
    # compute number of nodes
    nodes = len(set(from_pages) | set(to_pages))
    
    # add a new edge Y -> X
    from_pages.append(nodes)
    to_pages.append(nodes + 2)
    
    # add a new edge Z -> X
    from_pages.append(nodes + 1)
    to_pages.append(nodes + 2)
    
    # add links from Y (or Z) to older, popular pages
    for node in top_100:
        from_pages.append(nodes + 1)
        to_pages.append(node)

    # increase number of nodes and edges as needed
    nodes += 3
    edges += 102
    
    # fill the connectivity matrix, adjacency matrix. A sparse structure 
    # is used. That way the implementation is extremely memory efficient.
    csr_m = sparse.csr_matrix(([1]*edges,
                               (to_pages, from_pages)),
                              shape=(nodes, nodes))
    # return the adjacency sparse matrix
    return csr_m

In [58]:
csr_m4 = load_dataset4("./Dataset/stanweb.dat", top_100)
pr4, t4, list_conv4 = pagerank_galgo(csr_m4)

csr_m5 = load_dataset5("./Dataset/stanweb.dat", top_100)
pr5, t4, list_conv5 = pagerank_galgo(csr_m5)

#### **Results**

When we add links from X to older, popular pages (top 100), the PageRank of X falls significantly, and when we add connections from Y (or Z) to older, popular pages (top 100), the PageRank of X falls even further. This makes sense because Y's (or Z's) PageRank is now distributed across more webpages. As a result, this is not an effective method of improving X's PageRank, because the PageRank of a webpage generally improves as the number of webpages connecting to that webpage increases.

In [59]:
# if we add links from X to older, popular pages
pageRank_Xd = np.asarray(pr4[csr_m4.shape[0] - 1]).ravel()[0]
print("If we add links from X to older, popular pages the PageRank of page X is:", pageRank_Xd)

# if we add links from Y to older, popular pages
pageRank_Xd2 = np.asarray(pr5[csr_m5.shape[0] - 1]).ravel()[0]
print("If we add links from Y to older, popular pages the PageRank of page X is:", pageRank_Xd2)

If we add links from X to older, popular pages the PageRank of page X is: 1.4400732985803644e-06
If we add links from Y to older, popular pages the PageRank of page X is: 9.912111253145325e-07


### Question 2e

Based on the previous questions, it is clear that boosting a webpage's PageRank is a difficult process. That does not, however, make it impossible. The PageRank of a webpage can be enhanced as more and more pages link to it. Of course, if those pages have a high PageRank, their contribution will be considerably greater. Nonetheless, the more pages that point to a page, the greater the PageRank of that page. Furthermore, links between such pages may be beneficial.